# Building a song recommender

In [ ]:
%matplotlib inline

import pandas
from sklearn.cross_validation import train_test_split
import numpy as np
import time
from sklearn.externals import joblib
import Recommenders as Recommenders
import Evaluation as Evaluation

# Load music data

In [46]:
#Read userid-songid-listen_count triplets
#This step might take time to download data from external sources
triplets_file = 'https://static.turi.com/datasets/millionsong/10000.txt'
songs_metadata_file = 'https://static.turi.com/datasets/millionsong/song_data.csv'

song_df_1 = pandas.read_table(triplets_file,header=None)
song_df_1.columns = ['user_id', 'song_id', 'listen_count']

#Read song  metadata
song_df =  pandas.read_csv("/resources/data/samples/subject_data.csv")

#Merge the two dataframes above to create input dataframe for recommender systems
#song_df = pandas.merge(song_df_1, song_df_2.drop_duplicates(['song_id']), on="song_id", how="left") 
print ("success")

success


# Explore data

Music data shows how many times a user listened to a song, as well as the details of the song.

In [47]:
song_df.head()

,user_id,song_id,click_count,title,Unnamed: 4
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,Science,NaN
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOJPFQG12A58A7833A,2,Physics,NaN
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Chemistry,NaN
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Nuclear,NaN
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Maths,NaN


## Length of the dataset

In [48]:
len(song_df)

50

## Create a subset of the dataset

In [51]:
#song_df = song_df.head(10000)

#Merge song title and artist_name columns to make a merged column
song_df['song'] = song_df['title'].map(str) #+ " - " + song_df['click_count']

## Showing the most popular songs in the dataset

In [53]:
song_grouped = song_df.groupby(['song']).agg({'click_count': 'count'}).reset_index()
#print (song_grouped)
grouped_sum = song_grouped['click_count'].sum()
#print (grouped_sum)

song_grouped['percentage']  = song_grouped['click_count'].div(grouped_sum)*100
song_grouped.sort_values(['click_count', 'song'], ascending = [0,1])

,song,click_count,percentage
8,artificial intelligence,2,4.0
13,computers,2,4.0
17,english,2,4.0
20,hindi,2,4.0
21,history,2,4.0
24,literature,2,4.0
25,machine learning,2,4.0
38,social science,2,4.0
0,Chemistry,1,2.0
1,Maths,1,2.0


## Count number of unique users in the dataset

In [54]:
users = song_df['user_id'].unique()

In [55]:
len(users)

11

## Quiz 1. Count the number of unique songs in the dataset

In [56]:
###Fill in the code here
songs = song_df['song'].unique()
len(songs)

42

# Create a song recommender

In [57]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)
print(train_data.head(5))

                                     user_id             song_id  click_count  \
33  b80344d063b5ccb3212f76538f3d9e43d87dca9o  SOTRSFZ12A8C142BF6            1   
35  b80344d063b5ccb3212f76538f3d9e43d87dca9o  SOVHRGF12A8C13852F            1   
26  b80344d063b5ccb3212f76538f3d9e43d87dca9p  SODXRTY12AB0180F3B            1   
34  b80344d063b5ccb3212f76538f3d9e43d87dca9o  SOUKXIN12A8C133C7F            1   
18  b80344d063b5ccb3212f76538f3d9e43d87dca9z  SOKRIMP12A6D4F5DA3            1   

                      title  Unnamed: 4                     song  
33               mechanical         NaN               mechanical  
35                computers         NaN                computers  
26                  english         NaN                  english  
34                  thermal         NaN                  thermal  
18  artificial intelligence         NaN  artificial intelligence  


## Simple popularity-based recommender class (Can be used as a black box)

In [58]:
#Recommenders.popularity_recommender_py

### Create an instance of popularity based recommender class

In [59]:
pm = Recommenders.popularity_recommender_py()
pm.create(train_data, 'user_id', 'song')

### Use the popularity model to make some predictions

In [61]:
user_id = users[5]
pm.recommend(user_id)

,user_id,song,score,Rank
5,b80344d063b5ccb3212f76538f3d9e43d87dca9f,artificial intelligence,2,1.0
10,b80344d063b5ccb3212f76538f3d9e43d87dca9f,computers,2,2.0
13,b80344d063b5ccb3212f76538f3d9e43d87dca9f,english,2,3.0
15,b80344d063b5ccb3212f76538f3d9e43d87dca9f,hindi,2,4.0
20,b80344d063b5ccb3212f76538f3d9e43d87dca9f,machine learning,2,5.0
30,b80344d063b5ccb3212f76538f3d9e43d87dca9f,social science,2,6.0
0,b80344d063b5ccb3212f76538f3d9e43d87dca9f,Nuclear,1,7.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9f,Physics,1,8.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9f,Science,1,9.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9f,advance maths,1,10.0


### Quiz 2: Use the popularity based model to make predictions for the following user id (Note the difference in recommendations from the first user id).

In [62]:
###Fill in the code here
user_id = users[8]
pm.recommend(user_id)


,user_id,song,score,Rank
5,b80344d063b5ccb3212f76538f3d9e43d87dca9o,artificial intelligence,2,1.0
10,b80344d063b5ccb3212f76538f3d9e43d87dca9o,computers,2,2.0
13,b80344d063b5ccb3212f76538f3d9e43d87dca9o,english,2,3.0
15,b80344d063b5ccb3212f76538f3d9e43d87dca9o,hindi,2,4.0
20,b80344d063b5ccb3212f76538f3d9e43d87dca9o,machine learning,2,5.0
30,b80344d063b5ccb3212f76538f3d9e43d87dca9o,social science,2,6.0
0,b80344d063b5ccb3212f76538f3d9e43d87dca9o,Nuclear,1,7.0
1,b80344d063b5ccb3212f76538f3d9e43d87dca9o,Physics,1,8.0
2,b80344d063b5ccb3212f76538f3d9e43d87dca9o,Science,1,9.0
3,b80344d063b5ccb3212f76538f3d9e43d87dca9o,advance maths,1,10.0


## Build a song recommender with personalization

We now create an item similarity based collaborative filtering model that allows us to make personalized recommendations to each user. 

## Class for an item similarity based personalized recommender system (Can be used as a black box)

In [63]:
#Recommenders.item_similarity_recommender_py

### Create an instance of item similarity based recommender class

In [64]:
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song')

### Use the personalized model to make some song recommendations

In [65]:
#Print the songs for the user in training data
user_id = users[5]
user_items = is_model.get_user_items(user_id)
#
print("------------------------------------------------------------------------------------")
print("Training data songs for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id)

------------------------------------------------------------------------------------
Training data songs for the user userid: b80344d063b5ccb3212f76538f3d9e43d87dca9f:
------------------------------------------------------------------------------------
social science
history
politics
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
No. of unique songs for the user: 3
no. of unique songs in the training set: 34
Non zero values in cooccurence_matrix :12


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9f,arts,0.166667,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9f,hindi,0.111111,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9f,english,0.111111,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9f,medical,0.000000,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9f,botony,0.000000,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9f,Science,0.000000,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9f,Nuclear,0.000000,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9f,phisiology,0.000000,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9f,programming,0.000000,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9f,poetry,0.000000,10


### Quiz 3. Use the personalized model to make recommendations for the following user id. (Note the difference in recommendations from the first user id.)

In [66]:
user_id = users[7]
#Fill in the code here
user_items = is_model.get_user_items(user_id)
#
print("------------------------------------------------------------------------------------")
print("Training data songs for the user userid: %s:" % user_id)
print("------------------------------------------------------------------------------------")

for user_item in user_items:
    print(user_item)

print("----------------------------------------------------------------------")
print("Recommendation process going on:")
print("----------------------------------------------------------------------")

#Recommend songs for the user using personalized model
is_model.recommend(user_id)


------------------------------------------------------------------------------------
Training data songs for the user userid: b80344d063b5ccb3212f76538f3d9e43d87dca9p:
------------------------------------------------------------------------------------
english
hindi
social science
arts
----------------------------------------------------------------------
Recommendation process going on:
----------------------------------------------------------------------
No. of unique songs for the user: 4
no. of unique songs in the training set: 34
Non zero values in cooccurence_matrix :24


,user_id,song,score,rank
0,b80344d063b5ccb3212f76538f3d9e43d87dca9p,poetry,0.250,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9p,literature,0.250,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9p,english grammar,0.250,3
3,b80344d063b5ccb3212f76538f3d9e43d87dca9p,politics,0.125,4
4,b80344d063b5ccb3212f76538f3d9e43d87dca9p,history,0.125,5
5,b80344d063b5ccb3212f76538f3d9e43d87dca9p,medical,0.000,6
6,b80344d063b5ccb3212f76538f3d9e43d87dca9p,botony,0.000,7
7,b80344d063b5ccb3212f76538f3d9e43d87dca9p,Science,0.000,8
8,b80344d063b5ccb3212f76538f3d9e43d87dca9p,Nuclear,0.000,9
9,b80344d063b5ccb3212f76538f3d9e43d87dca9p,phisiology,0.000,10


### We can also apply the model to find similar songs to any song in the dataset

In [68]:
is_model.get_similar_items(['maths'])

no. of unique songs in the training set: 34
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,medical,0.0,1
1,,botony,0.0,2
2,,Science,0.0,3
3,,Nuclear,0.0,4
4,,phisiology,0.0,5
5,,programming,0.0,6
6,,poetry,0.0,7
7,,politics,0.0,8
8,,history,0.0,9
9,,it,0.0,10


### Quiz 4. Use the personalized recommender model to get similar songs for the following song.

In [69]:
song = 'Yellow - Coldplay'
###Fill in the code here
is_model.get_similar_items([song])

no. of unique songs in the training set: 34
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,medical,0.0,1
1,,botony,0.0,2
2,,Science,0.0,3
3,,Nuclear,0.0,4
4,,phisiology,0.0,5
5,,programming,0.0,6
6,,poetry,0.0,7
7,,politics,0.0,8
8,,history,0.0,9
9,,it,0.0,10


# Quantitative comparison between the models

We now formally compare the popularity and the personalized models using precision-recall curves. 

## Class to calculate precision and recall (This can be used as a black box)

In [70]:
#Evaluation.precision_recall_calculator

## Use the above precision recall calculator class to calculate the evaluation measures

In [71]:
start = time.time()

#Define what percentage of users to use for precision recall calculation
user_sample = 0.05

#Instantiate the precision_recall_calculator class
pr = Evaluation.precision_recall_calculator(test_data, train_data, pm, is_model)

#Call method to calculate precision and recall values
(pm_avg_precision_list, pm_avg_recall_list, ism_avg_precision_list, ism_avg_recall_list) = pr.calculate_measures(user_sample)

end = time.time()
print(end - start)

Length of user_test_and_training:4
Length of user sample:0


ZeroDivisionError: float division by zero

## Code to plot precision recall curve

In [ ]:
import pylab as pl

#Method to generate precision and recall curve
def plot_precision_recall(m1_precision_list, m1_recall_list, m1_label, m2_precision_list, m2_recall_list, m2_label):
    pl.clf()    
    pl.plot(m1_recall_list, m1_precision_list, label=m1_label)
    pl.plot(m2_recall_list, m2_precision_list, label=m2_label)
    pl.xlabel('Recall')
    pl.ylabel('Precision')
    pl.ylim([0.0, 0.20])
    pl.xlim([0.0, 0.20])
    pl.title('Precision-Recall curve')
    #pl.legend(loc="upper right")
    pl.legend(loc=9, bbox_to_anchor=(0.5, -0.2))
    pl.show()


In [ ]:
print("Plotting precision recall curves.")

plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")


### Generate Precision Recall curve using pickled results on a larger data subset(Python 3)

In [ ]:
print("Plotting precision recall curves for a larger subset of data (100,000 rows) (user sample = 0.005).")

#Read the persisted files 
pm_avg_precision_list = joblib.load('pm_avg_precision_list_3.pkl')
pm_avg_recall_list = joblib.load('pm_avg_recall_list_3.pkl')
ism_avg_precision_list = joblib.load('ism_avg_precision_list_3.pkl')
ism_avg_recall_list = joblib.load('ism_avg_recall_list_3.pkl')

print("Plotting precision recall curves.")
plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")

### Generate Precision Recall curve using pickled results on a larger data subset(Python 2.7)

In [ ]:
print("Plotting precision recall curves for a larger subset of data (100,000 rows) (user sample = 0.005).")

pm_avg_precision_list = joblib.load('pm_avg_precision_list_2.pkl')
pm_avg_recall_list = joblib.load('pm_avg_recall_list_2.pkl')
ism_avg_precision_list = joblib.load('ism_avg_precision_list_2.pkl')
ism_avg_recall_list = joblib.load('ism_avg_recall_list_2.pkl')

print("Plotting precision recall curves.")
plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")

The curve shows that the personalized model provides much better performance over the popularity model. 

# Matrix Factorization based Recommender System

### Import the required libraries

In [ ]:
#Code source written with help from: 
#http://antoinevastel.github.io/machine%20learning/python/2016/02/14/svd-recommender-system.html

import math as mt
import csv
from sparsesvd import sparsesvd #used for matrix factorization
import numpy as np
from scipy.sparse import csc_matrix #used for sparse matrix
from scipy.sparse.linalg import * #used for matrix multiplication

#Note: You may need to install the library sparsesvd. Documentation for 
#sparsesvd method can be found here:
#https://pypi.python.org/pypi/sparsesvd/

### Methods to compute SVD and recommendations

In [ ]:
#constants defining the dimensions of our User Rating Matrix (URM)
MAX_PID = 4
MAX_UID = 5

#Compute SVD of the user ratings matrix
def computeSVD(urm, K):
    U, s, Vt = sparsesvd(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(np.transpose(U), dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

#Compute estimated rating for the test user
def computeEstimatedRatings(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 

    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        #we convert the vector to dense format in order to get the indices 
        #of the movies with the best estimated ratings 
        estimatedRatings[userTest, :] = prod.todense()
        recom = (-estimatedRatings[userTest, :]).argsort()[:250]
    return recom


### Use SVD to make predictions for a test user id, say 4

In [ ]:
#Used in SVD calculation (number of latent factors)
K=2

#Initialize a sample user rating matrix
urm = np.array([[3, 1, 2, 3],[4, 3, 4, 3],[3, 2, 1, 5], [1, 6, 5, 2], [5, 0,0 , 0]])
urm = csc_matrix(urm, dtype=np.float32)

#Compute SVD of the input user ratings matrix
U, S, Vt = computeSVD(urm, K)

#Test user set as user_id 4 with ratings [0, 0, 5, 0]
uTest = [4]
print("User id for whom recommendations are needed: %d" % uTest[0])

#Get estimated rating for test user
print("Predictied ratings:")
uTest_recommended_items = computeEstimatedRatings(urm, U, S, Vt, uTest, K, True)
print(uTest_recommended_items)

### Quiz 4

## Understanding Intuition behind SVD

In [ ]:
%matplotlib inline
from pylab import *

#Plot all the users
print("Matrix Dimensions for U")
print(U.shape)

for i in range(0, U.shape[0]):
    plot(U[i,0], U[i,1], marker = "*", label="user"+str(i))

for j in range(0, Vt.T.shape[0]):
    plot(Vt.T[j,0], Vt.T[j,1], marker = 'd', label="item"+str(j))    
    
legend(loc="upper right")
title('User vectors in the Latent semantic space')
ylim([-0.7, 0.7])
xlim([-0.7, 0])
show()